In [1]:
import pandas as pd

movies = pd.read_csv("/data/ml-latest/movies.csv")

In [2]:
rows = []
for _, row in movies.iterrows():
    for genre in row["genres"].split("|"):
        rows.append([genre, row["movieId"]])

movies_genres = pd.DataFrame(rows, columns=["genre", "movieId"])
movies_genres["genre_id"] = movies_genres["genre"].astype("category").cat.codes.copy()
movies_genres.head()

,genre,movieId,genre_id
0,Adventure,1,2
1,Animation,1,3
2,Children,1,4
3,Comedy,1,5
4,Fantasy,1,9


In [3]:
ratings = pd.read_csv("/data/ml-latest/ratings.csv")

In [4]:
from scipy.sparse import coo_matrix
import numpy as np

user_item_matrix = coo_matrix(
    ((ratings["rating"] >= 4).astype(np.float32),
    (ratings["userId"], ratings["movieId"])),
    shape=(
        ratings["userId"].unique().max() + 1,
        movies["movieId"].unique().max() + 1
    )
)
user_item_matrix.eliminate_zeros()

In [5]:
import numpy as np

# делим разреженную матрицу на обучающую и тестовую
total_len = user_item_matrix.data.size
train_len = int(total_len * 0.8)
all_indices = np.arange(total_len)
np.random.seed(42)
train_indices = np.random.choice(all_indices, train_len, replace=False)
train_mask = np.in1d(all_indices, train_indices)

In [6]:
from scipy.sparse import coo_matrix

def get_masked(arr, mask):
    return coo_matrix(
        (
            [np.float32(item) for item in arr.data[mask]],
            (arr.row[mask], arr.col[mask])
        ),
        arr.shape
    )

In [7]:
train = get_masked(user_item_matrix, train_mask)
test = get_masked(user_item_matrix, ~train_mask)

In [8]:
from scipy.sparse import save_npz

save_npz("/data/other/lightfm_train.npz", train)
save_npz("/data/other/lightfm_test.npz", test)

In [9]:
from lightfm import LightFM

fm = LightFM()

In [10]:
%%time
fm.fit(
    interactions=train,
    epochs=100,
    num_threads=4,
    verbose=True
)

Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50
Epoch 51
Epoch 52
Epoch 53
Epoch 54
Epoch 55
Epoch 56
Epoch 57
Epoch 58
Epoch 59
Epoch 60
Epoch 61
Epoch 62
Epoch 63
Epoch 64
Epoch 65
Epoch 66
Epoch 67
Epoch 68
Epoch 69
Epoch 70
Epoch 71
Epoch 72
Epoch 73
Epoch 74
Epoch 75
Epoch 76
Epoch 77
Epoch 78
Epoch 79
Epoch 80
Epoch 81
Epoch 82
Epoch 83
Epoch 84
Epoch 85
Epoch 86
Epoch 87
Epoch 88
Epoch 89
Epoch 90
Epoch 91
Epoch 92
Epoch 93
Epoch 94
Epoch 95
Epoch 96
Epoch 97
Epoch 98
Epoch 99
CPU times: user 22min 55s, sys: 5.38 s, total: 23min
Wall time: 6min 28s


In [11]:
fm.get_params()

{'loss': 'logistic',
 'learning_schedule': 'adagrad',
 'no_components': 10,
 'learning_rate': 0.05,
 'k': 5,
 'n': 10,
 'rho': 0.95,
 'epsilon': 1e-06,
 'max_sampled': 10,
 'item_alpha': 0.0,
 'user_alpha': 0.0,
 'random_state': <mtrand.RandomState at 0x7f7e9d4aecf0>}

In [12]:
%%time
from lightfm.evaluation import reciprocal_rank

rr = reciprocal_rank(
    model=fm,
    test_interactions=test,
    train_interactions=train,
    num_threads=4
)

CPU times: user 46min 2s, sys: 15.2 s, total: 46min 17s
Wall time: 11min 39s


In [13]:
rr.mean()

0.22554931

In [14]:
%%time
fm.fit_partial(
    interactions=train,
    epochs=100,
    num_threads=4,
    verbose=True
)

Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50
Epoch 51
Epoch 52
Epoch 53
Epoch 54
Epoch 55
Epoch 56
Epoch 57
Epoch 58
Epoch 59
Epoch 60
Epoch 61
Epoch 62
Epoch 63
Epoch 64
Epoch 65
Epoch 66
Epoch 67
Epoch 68
Epoch 69
Epoch 70
Epoch 71
Epoch 72
Epoch 73
Epoch 74
Epoch 75
Epoch 76
Epoch 77
Epoch 78
Epoch 79
Epoch 80
Epoch 81
Epoch 82
Epoch 83
Epoch 84
Epoch 85
Epoch 86
Epoch 87
Epoch 88
Epoch 89
Epoch 90
Epoch 91
Epoch 92
Epoch 93
Epoch 94
Epoch 95
Epoch 96
Epoch 97
Epoch 98
Epoch 99
CPU times: user 24min 49s, sys: 16.3 s, total: 25min 5s
Wall time: 7min


In [15]:
%%time
rr = reciprocal_rank(
    model=fm,
    test_interactions=test,
    train_interactions=train,
    num_threads=4
)

CPU times: user 51min 37s, sys: 46.1 s, total: 52min 23s
Wall time: 15min 12s


In [16]:
rr.mean()

0.22613849

In [17]:
user_factors = fm.get_user_representations()
print(user_factors)

(array([0.        , 0.47650877, 0.25605968, ..., 0.6782449 , 0.07387851,
       1.1638339 ], dtype=float32), array([[-0.04659327,  0.02314085, -0.02466895, ..., -0.03308704,
        -0.00065173,  0.00602057],
       [-0.004812  , -0.07371379, -0.11643136, ...,  0.07905789,
         0.19510971,  0.11086403],
       [-0.00187309, -0.03050687, -0.01055165, ...,  0.00494255,
         0.06962708, -0.01133388],
       ...,
       [ 0.00532177, -0.10416693, -0.23676482, ...,  0.03437017,
         0.3238062 ,  0.17519532],
       [-0.04526329, -0.04749487,  0.01773976, ...,  0.00543904,
         0.05636593,  0.03587709],
       [-0.13312179, -0.11539035, -0.23195855, ..., -0.00137868,
         0.1807699 ,  0.18841772]], dtype=float32))


In [18]:
print(len(user_factors[0]))
print(user_factors[1].shape)

270897
(270897, 10)


In [19]:
item_factors = fm.get_item_representations()
print(item_factors)

(array([0.      , 9.500524, 8.182288, ..., 0.      , 0.      , 0.      ],
      dtype=float32), array([[ 4.7702733e-02,  2.4971811e-02, -2.3519304e-02, ...,
        -3.2393828e-02,  1.1656718e-02,  3.2342961e-03],
       [ 2.7348870e-01, -4.4326872e-01, -7.3448217e-01, ...,
         2.5944691e-02,  1.0534886e+00,  5.5204588e-01],
       [ 1.6300339e-01, -4.6971494e-01, -6.8998563e-01, ...,
         9.1063067e-02,  1.0339860e+00,  5.0601000e-01],
       ...,
       [ 4.3305378e-02,  2.8551701e-02, -3.2282114e-02, ...,
        -3.6246043e-02,  2.8052551e-04, -2.6937529e-02],
       [ 3.5948105e-02, -3.3759650e-02,  4.3549608e-02, ...,
        -1.5161917e-02,  2.6725572e-02,  4.1548215e-02],
       [ 9.6788481e-03,  3.2265585e-02, -2.3470962e-02, ...,
        -5.0364211e-03,  4.6508513e-02, -3.3635944e-02]], dtype=float32))


In [20]:
print(len(item_factors[0]))
print(item_factors[1].shape)

176280
(176280, 10)


In [21]:
from scipy.sparse import identity, hstack

item_feature_matrix = hstack([
    coo_matrix(
        (np.ones(movies_genres.count()[0], dtype=np.float32),
        (movies_genres["movieId"], movies_genres["genre_id"])),
        shape=(user_item_matrix.shape[1], movies_genres["genre_id"].unique().size)
    ),
    identity(user_item_matrix.shape[1])
])
item_feature_matrix.shape

(176280, 176300)

In [22]:
%%time
fm.fit(
    interactions=train,
    epochs=100,
    item_features=item_feature_matrix,
    num_threads=4,
    verbose=True
)

Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50
Epoch 51
Epoch 52
Epoch 53
Epoch 54
Epoch 55
Epoch 56
Epoch 57
Epoch 58
Epoch 59
Epoch 60
Epoch 61
Epoch 62
Epoch 63
Epoch 64
Epoch 65
Epoch 66
Epoch 67
Epoch 68
Epoch 69
Epoch 70
Epoch 71
Epoch 72
Epoch 73
Epoch 74
Epoch 75
Epoch 76
Epoch 77
Epoch 78
Epoch 79
Epoch 80
Epoch 81
Epoch 82
Epoch 83
Epoch 84
Epoch 85
Epoch 86
Epoch 87
Epoch 88
Epoch 89
Epoch 90
Epoch 91
Epoch 92
Epoch 93
Epoch 94
Epoch 95
Epoch 96
Epoch 97
Epoch 98
Epoch 99
CPU times: user 42min 39s, sys: 25.2 s, total: 43min 5s
Wall time: 13min 1s


In [23]:
new_item_factors = fm.get_item_representations()
print(new_item_factors)

(array([7.1033974, 8.571053 , 6.9686017, ..., 0.       , 0.       ,
       0.       ], dtype=float32), array([[ 0.12428182, -0.1664608 ,  0.01275694, ...,  0.01781604,
        -0.18183285,  0.2846708 ],
       [ 0.01522175,  0.02957721, -0.03500439, ..., -0.00392613,
         0.02922761,  0.0489003 ],
       [-0.02990988,  0.02416395, -0.0543372 , ..., -0.03478535,
         0.01524367, -0.01387728],
       ...,
       [ 0.04079019,  0.02935628,  0.03091659, ..., -0.0268052 ,
        -0.03193732,  0.04339782],
       [-0.03863824,  0.00103276,  0.03795191, ..., -0.01073747,
         0.02687897, -0.00703392],
       [ 0.00216349,  0.02455151,  0.01694351, ...,  0.01938514,
         0.02024632,  0.03958329]], dtype=float32))


In [24]:
print(len(new_item_factors[0]))
print(new_item_factors[1].shape)

176300
(176300, 10)


In [25]:
%%time
rr = reciprocal_rank(
    model=fm,
    test_interactions=test,
    train_interactions=train,
    item_features=item_feature_matrix,
    num_threads=4
)

CPU times: user 1h 6min 23s, sys: 56.5 s, total: 1h 7min 20s
Wall time: 17min


In [26]:
rr.mean()

0.0030269846